### SELECT statement

1.Bigger than Russia
List each country name where the population is larger than that of 'Russia'.

In [ ]:
SELECT name FROM world
  WHERE population >
     (SELECT population FROM world
      WHERE name='Russia')

2.
Show the countries in Europe with a per capita GDP greater than 'United Kingdom'.

In [ ]:
SELECT name
FROM world
WHERE continent = 'Europe' AND gdp/population > (SELECT gdp/population
                                                FROM world 
                                                WHERE name = 'United Kingdom')

3. List the name and continent of countries in the continents containing either Argentina or Australia. Order by name of the country.

In [ ]:
SELECT name, continent
FROM world
WHERE continent IN (SELECT continent
                    FROM world
                    WHERE name = 'Argentina' OR name = 'Australia')
ORDER BY name

4. Which country has a population that is more than Canada but less than Poland? Show the name and the population.


In [ ]:
SELECT name, population
FROM world
WHERE population >  (SELECT population
                    FROM world
                    WHERE name = 'Canada')
AND population < (SELECT population
                  FROM world
                  WHERE name = 'Poland')

5.Germany (population 80 million) has the largest population of the countries in Europe. Austria (population 8.5 million) has 11% of the population of Germany.

Show the name and the population of each country in Europe. Show the population as a percentage of the population of Germany.

In [ ]:
SELECT name, CONCAT(ROUND(100*population/(SELECT population 
                             FROM world
                             WHERE name ='Germany')), '%') as percentage
FROM world
WHERE continent = 'EUROPE'

6.
Which countries have a GDP greater than every country in Europe? [Give the name only.] (Some countries may have NULL gdp values)

In [ ]:
SELECT name
FROM world
WHERE gdp > ALL (SELECT gdp
                 FROM world
                 WHERE continent = 'Europe' AND gdp > 0)

7.
Find the largest country (by area) in each continent, show the continent, the name and the area:

In [ ]:
SELECT continent, name, area FROM world x
WHERE area >= ALL
    (SELECT area 
     FROM world y
     WHERE y.continent=x.continent AND population>0)

8.
List each continent and the name of the country that comes first alphabetically.

In [ ]:
SELECT continent, name
FROM world x
WHERE name <= ALL
    (SELECT name FROM world y
    WHERE y.continent=x.continent
    AND population>0)

9.
Find the continents where all countries have a population <= 25000000. Then find the names of the countries associated with these continents. Show name, continent and population.

In [ ]:
SELECT  name, continent, population
FROM world x
WHERE 25000000 > ALL
(SELECT population
FROM world y
WHERE x.continent = y.continent AND population >0)

10.
Some countries have populations more than three times that of all of their neighbours (in the same continent). Give the countries and continents.

In [ ]:
SELECT name, continent
FROM world x
WHERE population > ALL
    (SELECT 3*population
    FROM world y
    WHERE x.continent = y.continent AND population >0 AND y.name != x.name)



### JOIN Statements


The first example shows the goal scored by a player with the last name 'Bender'. The * says to list all the columns in the table - a shorter way of saying matchid, teamid, player, gtime
Modify it to show the matchid and player name for all goals scored by Germany. To identify German players, check for: teamid = 'GER'

In [1]:
SELECT matchid, player  
FROM goal 
WHERE teamid = 'GER'

SyntaxError: invalid syntax (3435229840.py, line 1)

2.
From the previous query you can see that Lars Bender's scored a goal in game 1012. Now we want to know what teams were playing in that match.
Notice in the that the column matchid in the goal table corresponds to the id column in the game table. We can look up information about game 1012 by finding that row in the game table.
Show id, stadium, team1, team2 for just game 1012

In [ ]:
SELECT id,stadium,team1,team2
  FROM game 
WHERE id = 1012


3 You can combine the two steps into a single query with a JOIN.

SELECT *
  FROM game JOIN goal ON (id=matchid)
The FROM clause says to merge data from the goal table with that from the game table. The ON says how to figure out which rows in game go with which rows in goal - the matchid from goal must match id from game. (If we wanted to be more clear/specific we could say
ON (game.id=goal.matchid)

The code below shows the player (from the goal) and stadium name (from the game table) for every goal scored.

Modify it to show the player, teamid, stadium and mdate for every German goal.


In [ ]:
SELECT player,teamid, stadium, mdate
  FROM game JOIN goal ON (id=matchid)
WHERE teamid = 'GER'

4. Use the same JOIN as in the previous question.

Show the team1, team2 and player for every goal scored by a player called Mario player LIKE 'Mario%'

In [ ]:
SELECT team1, team2, player
FROM game JOIN goal ON (id = matchid)
WHERE player LIKE 'Mario%'

5. The table eteam gives details of every national team including the coach. You can JOIN goal to eteam using the phrase goal JOIN eteam on teamid=id
Show player, teamid, coach, gtime for all goals scored in the first 10 minutes gtime<=10

In [ ]:
SELECT player, teamid, coach, gtime
FROM goal JOIN eteam ON (teamid=id)
WHERE goal.gtime<=10


6. To JOIN game with eteam you could use either
game JOIN eteam ON (team1=eteam.id) or game JOIN eteam ON (team2=eteam.id)

Notice that because id is a column name in both game and eteam you must specify eteam.id instead of just id

List the dates of the matches and the name of the team in which 'Fernando Santos' was the team1 coach.

In [ ]:
SELECT mdate, teamname
FROM game JOIN eteam ON (team1 = eteam.id)
WHERE eteam.coach = 'Fernando Santos' 

7. List the player for every goal scored in a game where the stadium was 'National Stadium, Warsaw'


In [ ]:
SELECT player
FROM goal JOIN game ON (id = matchid)
WHERE stadium = 'National Stadium, Warsaw'

8. The example query shows all goals scored in the Germany-Greece quarterfinal.
Instead show the name of all players who scored a goal against Germany.

In [ ]:
SELECT DISTINCT(player)
FROM game JOIN goal ON matchid = id 
WHERE (team1 = 'GER' OR team2='GER') AND teamid <> 'GER' 

9. Show teamname and the total number of goals scored.

In [ ]:
COUNT and GROUP BY
SELECT teamname, COUNT(player)
  FROM eteam JOIN goal ON id=teamid
GROUP BY teamname

10. Show the stadium and the number of goals scored in each stadium.


In [ ]:
SELECT stadium, COUNT(player)
  FROM game JOIN goal ON id=matchid
GROUP BY stadium

11 . For every match involving 'POL', show the matchid, date and the number of goals scored.

In [4]:
SELECT matchid, mdate, COUNT(teamid)
FROM game JOIN goal ON matchid = id 
WHERE (team1 = 'POL' OR team2 = 'POL')
GROUP BY matchid, mdate

SyntaxError: invalid syntax (442417754.py, line 1)

12. For every match where 'GER' scored, show matchid, match date and the number of goals scored by 'GER'

In [ ]:
SELECT matchid, mdate, COUNT(player)
FROM game JOIN goal ON matchid = id 
WHERE (team1 = 'GER' OR team2 = 'GER') AND (teamid = 'GER')
GROUP BY matchid, mdate

13.List every match with the goals scored by each team as shown. This will use "CASE WHEN" which has not been explained in any previous exercises.
mdate	team1	score1	team2	score2
1 July 2012	ESP	4	ITA	0
10 June 2012	ESP	1	ITA	1
10 June 2012	IRL	1	CRO	3
...
Notice in the query given every goal is listed. If it was a team1 goal then a 1 appears in score1, otherwise there is a 0. You could SUM this column to get a count of the goals scored by team1. Sort your result by mdate, matchid, team1 and team2.


In [ ]:
SELECT mdate, 
team1, SUM(CASE WHEN teamid=team1 THEN 1 ELSE 0 END) AS score1,
team2, SUM(CASE WHEN teamid=team2 THEN 1 ELSE 0 END) AS score2
FROM game JOIN goal ON matchid = id
GROUP BY matchid, mdate, team1, team2
ORDER BY mdate, matchid, team1 , team2

